In [ ]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker  # Ensure this is the correct import for your Tracker class

# Initialize YOLO model
model = YOLO('yolov8x.pt')

# Vehicle-related classes (as per YOLO)
vehicle_classes = ['car', 'motorcycle', 'bus', 'truck']

# Emergency vehicles (Checking detected names)
emergency_classes = ['ambulance', 'fire truck', 'police', 'truck', 'bus']  # YOLO may detect as truck/bus

# Initialize Tracker
tracker = Tracker()
count = 0

# Open video file
cap = cv2.VideoCapture('traffic_with_emergency.mp4')

# Dictionaries to track vehicles crossing lines
down = {}
up = {}
counter_down = []
counter_up = []

# Flags to track emergency vehicle crossings
emergency_crossed_red = False  # Track if an emergency vehicle has crossed the red line
emergency_crossed_blue = False  # Track if an emergency vehicle has crossed the blue line

# Initialize the three-lane window
lane_window = np.zeros((500, 1020, 3), dtype=np.uint8)
lane_window[:] = (50, 50, 50)  # Background color (dark gray)

# Define lane properties
num_lanes = 3
lane_width = 300
lane_spacing = 20
lane_start_x = 100
lane_end_x = lane_start_x + lane_width
lane_y = 50  # Starting y-coordinate for lanes
lane_height = 400

# Draw lanes
for i in range(num_lanes):
    lane_x = lane_start_x + i * (lane_width + lane_spacing)
    cv2.rectangle(lane_window, (lane_x, lane_y), (lane_x + lane_width, lane_y + lane_height), (255, 255, 255), 2)
    # Initialize signals to green
    signal_color = (0, 255, 0)  # Green
    cv2.rectangle(lane_window, (lane_x + lane_width - 30, lane_y + 10), (lane_x + lane_width - 10, lane_y + 30), signal_color, -1)
    cv2.putText(lane_window, 'Lane {}'.format(i+1), (lane_x + 10, lane_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

# Initialize signal states for each lane
lane_signals = {
    'lane1': 'green',
    'lane2': 'green',
    'lane3': 'green'
}

# Flag to track if the leftmost lane should be red due to emergency vehicle
leftmost_lane_red = False

def determine_lane(cx, lane_start_x, lane_width, lane_spacing):
    for i in range(num_lanes):
        lane_x = lane_start_x + i * (lane_width + lane_spacing)
        if lane_x <= cx <= lane_x + lane_width:
            return i+1
    return None

while True:    
    ret, frame = cap.read()
    if not ret:
        break
    
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    detections = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(detections).astype("float")

    detected_vehicles = []
    for _, row in px.iterrows():
        x1, y1, x2, y2, _, class_id = map(int, row[:6])
        vehicle_type = model.names[class_id]  # Get detected class name
        
        if vehicle_type in vehicle_classes or vehicle_type in emergency_classes:
            detected_vehicles.append([x1, y1, x2, y2, vehicle_type])  # Store vehicle type

    bbox_id = tracker.update([box[:4] for box in detected_vehicles])  # Track using only bbox coords

    for i, bbox in enumerate(bbox_id):
        x3, y3, x4, y4, obj_id = bbox
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)

        red_line_y = 198
        blue_line_y = 400
        offset = 7

        # Red line logic
        if red_line_y - offset < cy < red_line_y + offset:
            down[obj_id] = cy   

        if obj_id in down:
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, str(obj_id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            counter_down.append(obj_id)

            # Check if the vehicle that crossed is an emergency vehicle
            if detected_vehicles[i][4] in emergency_classes:
                leftmost_lane_red = True

        # Blue line logic
        if blue_line_y - offset < cy < blue_line_y + offset:
            up[obj_id] = cy   

        if obj_id in up:
            cv2.circle(frame, (cx, cy), 4, (255, 0, 0), -1)
            cv2.putText(frame, str(obj_id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            counter_up.append(obj_id)

            # Check if the vehicle that crossed is an emergency vehicle
            if detected_vehicles[i][4] in emergency_classes:
                leftmost_lane_red = False  # Reset when emergency vehicle leaves

        # Display vehicle name on top
        vehicle_name = detected_vehicles[i][4]  # Get vehicle type
        cv2.putText(frame, vehicle_name, (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    total_vehicles = len(set(counter_down))
    
    # Signal Logic: Green if emergency vehicle crossed red line, Red if it crossed blue line
    if leftmost_lane_red:
        lane_signals['lane1'] = 'red'
    else:
        lane_signals['lane1'] = 'green'

    # Update lane signals based on emergency vehicle presence
    for lane in ['lane2', 'lane3']:
        lane_signals[lane] = 'green'

    # Draw signals for each lane
    for i in range(num_lanes):
        lane = f'lane{i+1}'
        lane_x = lane_start_x + i * (lane_width + lane_spacing)
        signal_color = (0, 255, 0)  # Default green
        if lane_signals[lane] == 'red':
            signal_color = (0, 0, 255)  # Red
        cv2.rectangle(lane_window, (lane_x + lane_width - 30, lane_y + 10), (lane_x + lane_width - 10, lane_y + 30), signal_color, -1)
        if lane_signals[lane] == 'red':
            cv2.putText(lane_window, 'Emergency', (lane_x + 10, lane_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)
        else:
            cv2.putText(lane_window, ' ', (lane_x + 10, lane_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Draw signal at the center (No Text)
    signal_color_main = (0, 0, 255)  # Default red
    if leftmost_lane_red:
        signal_color_main = (0, 255, 0)  # Green
    cv2.rectangle(frame, (470, 20), (520, 70), signal_color_main, -1)

    # Draw red line
    cv2.line(frame, (222, red_line_y), (824, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'Red Line', (222, red_line_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Draw blue line
    cv2.line(frame, (172, blue_line_y), (774, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'Blue Line', (172, blue_line_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Vehicle count display
    cv2.putText(frame, f'Vehicles: {total_vehicles}', (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the three-lane window
    cv2.imshow("Three-Lane Monitoring", lane_window)

    # Display the main traffic monitoring window
    cv2.imshow("Traffic Monitoring", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 14 cars, 2 buss, 3 trucks, 39.2ms
Speed: 1.8ms preprocess, 39.2ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 14 cars, 2 buss, 2 trucks, 14.1ms
Speed: 2.2ms preprocess, 14.1ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 14 cars, 2 buss, 1 truck, 14.1ms
Speed: 2.3ms preprocess, 14.1ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 cars, 3 buss, 14.1ms
Speed: 2.2ms preprocess, 14.1ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 cars, 3 buss, 1 truck, 14.1ms
Speed: 2.5ms preprocess, 14.1ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 cars, 2 buss, 2 trucks, 14.1ms
Speed: 2.1ms preprocess, 14.1ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 cars, 2 buss, 1 truck, 14.5ms
Speed: 2.2ms preprocess, 14.5ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 

In [2]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker  # Ensure this is the correct import for your Tracker class
from lime.lime_image import LimeImageExplainer  # Import LIME for image explanations
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries  # For visualizing LIME explanations

# Initialize YOLO model
model = YOLO('yolov8x.pt')

# Vehicle-related classes (as per YOLO)
vehicle_classes = ['car', 'motorcycle', 'bus', 'truck']

# Emergency vehicles (Checking detected names)
emergency_classes = ['ambulance', 'fire truck', 'police', 'truck', 'bus']  # YOLO may detect as truck/bus

# Initialize Tracker
tracker = Tracker()
count = 0

# Open video file
cap = cv2.VideoCapture('traffic_with_emergency.mp4')

# Dictionaries to track vehicles crossing lines
down = {}
up = {}
counter_down = []
counter_up = []

# Flags to track emergency vehicle crossings
emergency_crossed_red = False
emergency_crossed_blue = False

# Initialize the three-lane window
lane_window = np.zeros((500, 1020, 3), dtype=np.uint8)
lane_window[:] = (50, 50, 50)  # Background color (dark gray)

# Define lane properties
num_lanes = 3
lane_width = 300
lane_spacing = 20
lane_start_x = 100
lane_end_x = lane_start_x + lane_width
lane_y = 50  # Starting y-coordinate for lanes
lane_height = 400

# Draw lanes
for i in range(num_lanes):
    lane_x = lane_start_x + i * (lane_width + lane_spacing)
    cv2.rectangle(lane_window, (lane_x, lane_y), (lane_x + lane_width, lane_y + lane_height), (255, 255, 255), 2)
    signal_color = (0, 255, 0)  # Green
    cv2.rectangle(lane_window, (lane_x + lane_width - 30, lane_y + 10), (lane_x + lane_width - 10, lane_y + 30), signal_color, -1)
    cv2.putText(lane_window, 'Lane {}'.format(i+1), (lane_x + 10, lane_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

# Initialize signal states for each lane
lane_signals = {
    'lane1': 'green',
    'lane2': 'green',
    'lane3': 'green'
}

# Flag to track if the leftmost lane should be red due to emergency vehicle
leftmost_lane_red = False

# Initialize LIME explainer
lime_explainer = LimeImageExplainer()

# Define a prediction function for LIME
def predict_fn(images):
    """
    Prediction function for LIME to explain YOLOv8x predictions.
    Input: List of images (numpy arrays in RGB format)
    Output: Probabilities for each class (normal vehicle vs. emergency vehicle)
    """
    predictions = []
    for img in images:
        # Convert RGB to BGR for YOLO
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        # Resize to match the input size expected by YOLO
        img_resized = cv2.resize(img_bgr, (1020, 500))
        # Predict using YOLO
        results = model.predict(img_resized)
        detections = results[0].boxes.data.detach().cpu().numpy()
        px = pd.DataFrame(detections).astype("float")
        
        # Initialize probabilities (normal vehicle: 0, emergency vehicle: 1)
        probs = np.zeros(2)  # [normal, emergency]
        for _, row in px.iterrows():
            x1, y1, x2, y2, _, class_id = map(int, row[:6])
            vehicle_type = model.names[class_id]
            if vehicle_type in emergency_classes:
                probs[1] = 1.0  # Emergency vehicle detected
                break
            else:
                probs[0] = 1.0  # Normal vehicle
        predictions.append(probs)
    return np.array(predictions)

def determine_lane(cx, lane_start_x, lane_width, lane_spacing):
    for i in range(num_lanes):
        lane_x = lane_start_x + i * (lane_width + lane_spacing)
        if lane_x <= cx <= lane_x + lane_width:
            return i+1
    return None

# Variable to store the frame for LIME explanation
frame_for_lime = None
explain_lime = False

while True:    
    ret, frame = cap.read()
    if not ret:
        break
    
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    # Store the frame for LIME explanation if needed
    if explain_lime:
        frame_for_lime = frame.copy()

    results = model.predict(frame)
    detections = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(detections).astype("float")

    detected_vehicles = []
    for _, row in px.iterrows():
        x1, y1, x2, y2, _, class_id = map(int, row[:6])
        vehicle_type = model.names[class_id]  # Get detected class name
        
        if vehicle_type in vehicle_classes or vehicle_type in emergency_classes:
            detected_vehicles.append([x1, y1, x2, y2, vehicle_type])  # Store vehicle type

    bbox_id = tracker.update([box[:4] for box in detected_vehicles])  # Track using only bbox coords

    for i, bbox in enumerate(bbox_id):
        x3, y3, x4, y4, obj_id = bbox
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)

        red_line_y = 198
        blue_line_y = 400
        offset = 7

        # Red line logic
        if red_line_y - offset < cy < red_line_y + offset:
            down[obj_id] = cy   
            if detected_vehicles[i][4] in emergency_classes:
                leftmost_lane_red = True
                # Trigger LIME explanation for this emergency vehicle
                if not explain_lime:
                    explain_lime = True
                    frame_for_lime = frame.copy()

        if obj_id in down:
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, str(obj_id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            counter_down.append(obj_id)

        # Blue line logic
        if blue_line_y - offset < cy < blue_line_y + offset:
            up[obj_id] = cy   
            if detected_vehicles[i][4] in emergency_classes:
                leftmost_lane_red = False  # Reset when emergency vehicle leaves

        if obj_id in up:
            cv2.circle(frame, (cx, cy), 4, (255, 0, 0), -1)
            cv2.putText(frame, str(obj_id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            counter_up.append(obj_id)

        # Display vehicle name on top
        vehicle_name = detected_vehicles[i][4]  # Get vehicle type
        cv2.putText(frame, vehicle_name, (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    total_vehicles = len(set(counter_down))
    
    # Signal Logic: Green if emergency vehicle crossed red line, Red if it crossed blue line
    if leftmost_lane_red:
        lane_signals['lane1'] = 'red'
    else:
        lane_signals['lane1'] = 'green'

    # Update lane signals based on emergency vehicle presence
    for lane in ['lane2', 'lane3']:
        lane_signals[lane] = 'green'

    # Draw signals for each lane
    for i in range(num_lanes):
        lane = f'lane{i+1}'
        lane_x = lane_start_x + i * (lane_width + lane_spacing)
        signal_color = (0, 255, 0)  # Default green
        if lane_signals[lane] == 'red':
            signal_color = (0, 0, 255)  # Red
        cv2.rectangle(lane_window, (lane_x + lane_width - 30, lane_y + 10), (lane_x + lane_width - 10, lane_y + 30), signal_color, -1)
        if lane_signals[lane] == 'red':
            cv2.putText(lane_window, 'Emergency', (lane_x + 10, lane_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)
        else:
            cv2.putText(lane_window, ' ', (lane_x + 10, lane_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Draw signal at the center (No Text)
    signal_color_main = (0, 0, 255)  # Default red
    if leftmost_lane_red:
        signal_color_main = (0, 255, 0)  # Green
    cv2.rectangle(frame, (470, 20), (520, 70), signal_color_main, -1)

    # Draw red line
    cv2.line(frame, (222, red_line_y), (824, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'Red Line', (222, red_line_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Draw blue line
    cv2.line(frame, (172, blue_line_y), (774, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'Blue Line', (172, blue_line_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Vehicle count display
    cv2.putText(frame, f'Vehicles: {total_vehicles}', (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the three-lane window
    cv2.imshow("Three-Lane Monitoring", lane_window)

    # Display the main traffic monitoring window
    cv2.imshow("Traffic Monitoring", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop, if an emergency vehicle was detected, explain with LIME
if explain_lime and frame_for_lime is not None:
    print("Generating LIME explanation for emergency vehicle detection...")
    # Convert BGR to RGB for LIME
    frame_rgb = cv2.cvtColor(frame_for_lime, cv2.COLOR_BGR2RGB)
    # Explain the prediction
    explanation = lime_explainer.explain_instance(
        frame_rgb,
        predict_fn,
        top_labels=2,
        hide_color=0,
        num_samples=1000
    )
    # Visualize the explanation
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0],
        positive_only=True,
        num_features=5,
        hide_rest=False
    )
    # Mark the boundaries of the important regions
    img_boundaries = mark_boundaries(temp / 255.0, mask)
    # Save the explanation
    plt.imsave('lime_explanation.png', img_boundaries)
    print("LIME explanation saved as 'lime_explanation.png'")

cap.release()
cv2.destroyAllWindows()

c:\Users\nisha\anaconda3\envs\Paper\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



0: 320x640 1 person, 16 cars, 1 bus, 3 trucks, 53.3ms
Speed: 3.1ms preprocess, 53.3ms inference, 67.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 16 cars, 1 bus, 3 trucks, 28.9ms
Speed: 1.6ms preprocess, 28.9ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 16 cars, 2 buss, 2 trucks, 28.9ms
Speed: 1.3ms preprocess, 28.9ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 17 cars, 1 bus, 3 trucks, 28.7ms
Speed: 1.4ms preprocess, 28.7ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 17 cars, 1 bus, 3 trucks, 28.8ms
Speed: 1.4ms preprocess, 28.8ms inference, 1.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 16 cars, 1 bus, 3 trucks, 22.8ms
Speed: 1.2ms preprocess, 22.8ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 19 cars, 1 bus, 4 trucks, 22.8ms
Speed: 1.5ms preprocess, 22.8ms inference, 1.5ms po

  1%|          | 7/1000 [00:00<00:36, 26.91it/s]


0: 320x640 18 cars, 4 trucks, 198.6ms
Speed: 2.4ms preprocess, 198.6ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 21 cars, 2 motorcycles, 2 trucks, 1 traffic light, 59.3ms
Speed: 1.5ms preprocess, 59.3ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 17 cars, 1 motorcycle, 1 truck, 1 boat, 28.4ms
Speed: 0.9ms preprocess, 28.4ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 25 cars, 2 trucks, 1 traffic light, 28.5ms
Speed: 0.9ms preprocess, 28.5ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 18 cars, 2 trucks, 28.6ms
Speed: 1.0ms preprocess, 28.6ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 18 cars, 1 boat, 1 traffic light, 2 chairs, 28.6ms
Speed: 1.0ms preprocess, 28.6ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 24 cars, 1 truck, 1 traffic light, 26.5ms
Speed: 1

  2%|▏         | 19/1000 [00:01<01:03, 15.55it/s]

...


0: 320x640 27 cars, 1 truck, 1 boat, 1 traffic light, 1 fire hydrant, 22.5ms
Speed: 1.1ms preprocess, 22.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 24 cars, 1 motorcycle, 1 bus, 3 trucks, 1 traffic light, 22.4ms
Speed: 0.9ms preprocess, 22.4ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 24 cars, 3 motorcycles, 2 trucks, 1 traffic light, 22.5ms
Speed: 1.3ms preprocess, 22.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 persons, 22 cars, 1 traffic light, 22.5ms
Speed: 1.0ms preprocess, 22.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 27 cars, 1 traffic light, 22.6ms
Speed: 1.2ms preprocess, 22.6ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 23 cars, 1 truck, 2 traffic lights, 22.5ms
Speed: 1.2ms preprocess, 22.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 22 ca

 60%|█████▉    | 597/1000 [00:43<00:26, 15.15it/s]


0: 320x640 2 persons, 18 cars, 1 truck, 1 boat, 22.5ms
Speed: 1.1ms preprocess, 22.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 20 cars, 2 trucks, 1 traffic light, 22.5ms
Speed: 0.9ms preprocess, 22.5ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 22 cars, 2 traffic lights, 22.5ms
Speed: 0.9ms preprocess, 22.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 29 cars, 3 trucks, 1 traffic light, 22.5ms
Speed: 0.9ms preprocess, 22.5ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 23 cars, 1 truck, 1 traffic light, 22.4ms
Speed: 1.0ms preprocess, 22.4ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 21 cars, 1 motorcycle, 1 truck, 1 traffic light, 1 parking meter, 22.5ms
Speed: 1.0ms preprocess, 22.5ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 21 cars, 2 trucks, 3 traffic light

 60%|██████    | 603/1000 [00:44<00:30, 13.02it/s]

: 